In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.initializers import Constant
from itertools import product
from tqdm import tqdm


from Libs.load_data import ClassificationDataLoader, DataLoader, get_dataset_split
from Libs.threshold import get_labels_physic
from Libs.keras_f1score import f1_m

2023-03-11 16:38:40.776540: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-11 16:38:41.470247: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-11 16:38:41.470340: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-11 16:38:44.698281: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
data_loader = ClassificationDataLoader(run=100, N=1000, s=0.5, t=0.01, d=0.2, m=1)
params = data_loader.get_params()
params

{'run': 100,
 'sigma': [0.5],
 'theta': [0.01],
 'mu': [1],
 'delta': [0.2],
 'N': 1000}

In [5]:
Xs, best_labels = data_loader.load_data(override=True)

Loading Data


100%|██████████| 100/100 [00:00<00:00, 322.71it/s]

Loading Labels
Labels Loaded


In [6]:
Xs.shape, best_labels.shape

((100, 1000), (100, 1000))

In [7]:
bindexes = data_loader.get_standard_indexes()
df_train,df_val,df_test = get_dataset_split(Xs, best_labels, bindexes, window_size=20, overlap_size=19,
                                            label_treshold=1, split_on_run=True, shuffle_run=False, 
                                            shuffle_window=False, test_size = 0.3, val_size=0.2, 
                                            get_validation=True, random_state=42)

In [8]:
# number of classes
print('Training set:')
print(df_train['future_flare'].value_counts(), '\n')
pos = df_train['future_flare'].value_counts()[0]
true = df_train['future_flare'].value_counts()[1]
print('validation set:')
print(df_val['future_flare'].value_counts(), '\n')
print('Test set:')
print(df_test['future_flare'].value_counts(), '\n')

Training set:
0    43768
1     4252
Name: future_flare, dtype: int64 

validation set:
0    18233
1     2347
Name: future_flare, dtype: int64 

Test set:
0    26357
1     3043
Name: future_flare, dtype: int64 



In [9]:
X_train, y_train = df_train.iloc[:,:-1].to_numpy(), df_train.future_flare.to_numpy()
X_val, y_val = df_val.iloc[:,:-1].to_numpy(), df_val.future_flare.to_numpy()
X_test, y_test = df_test.iloc[:,:-1].to_numpy(), df_test.future_flare.to_numpy()
X = np.vstack((X_train, X_val, X_test))
y = np.hstack((y_train, y_val, y_test))
print('X ## Train:', X_train.shape, 'Val:', X_val.shape, 'Test:', X_test.shape)
print('y ## Train:', y_train.shape, 'Val:', y_val.shape, 'Test:', y_test.shape)

X ## Train: (48020, 20) Val: (20580, 20) Test: (29400, 20)
y ## Train: (48020,) Val: (20580,) Test: (29400,)


In [10]:
np.log(true/pos)

-2.3315134962818234

In [11]:
initial_bias = Constant([np.log(true/pos)])
# define model
model = Sequential()
model.add(Bidirectional(LSTM(20, activation='relu'), input_shape=(X_train.shape[1], 1)))
model.add(Dense(30, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid',bias_initializer=initial_bias))
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=[f1_m, 'accuracy'])

print(model.summary())

2023-03-11 16:40:53.192431: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-11 16:40:53.192637: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-11 16:40:53.192689: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (83dc2cdd3c94): /proc/driver/nvidia/version does not exist
2023-03-11 16:40:53.193367: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 40)               3520      
 l)                                                              
                                                                 
 dense (Dense)               (None, 30)                1230      
                                                                 
 dense_1 (Dense)             (None, 10)                310       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 5,071
Trainable params: 5,071
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
epochs = 20
batch_size = 32
# define callbacks
callbacks = [
    ModelCheckpoint(
        os.path.join("models", "LSTM_best_weights.h5"), save_weights_only=True, monitor="val_loss"
    ),
    EarlyStopping(monitor="val_loss", patience=3, verbose=1),
]
# fit model
model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=(X_val, y_val),
    verbose=1,
)

Epoch 1/20
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
1501/1501 [==============================] - 37s 22ms/step - loss: 0.0517 - f1_m: 0.8051 - accuracy: 0.9811 - val_loss: 0.0219 - val_f1_m: 0.1485 - val_accuracy: 0.9926
Epoch 2/20
1501/1501 [==============================] - 48s 32ms/step - loss: 0.0277 - f1_m: 0.8642 - accuracy: 0.9883 - val_loss: 0.0182 - val_f1_m: 0.1450 - val_accuracy: 0.9934
Epoch 3/20
1501/1501 [==============================] - 44s 29ms/step - loss: 0.0254 - f1_m: 0.8643 - accuracy: 0.9893 - val_loss: 0.0196 - val_f1_m: 0.1477 - val_accuracy: 0.9926
Epoch 4/20
1501/1501 [==============================] - 41s 28ms/step - loss: 0.0240 - f1_m: 0.8847 - accuracy: 0.9900 - val_loss: 0.0211 - val_f1_m: 0.1450 - val_accuracy: 0.9917
Epoch 5/20
1501/1501 [==============================] - 37s 25ms/step - loss: 0

In [13]:
scores = model.evaluate(X_test, y_test, verbose=0)
y_pred = np.round(model.predict(X_test), 0)

print("### Evaluation on test set ###")
print("Accuracy: %.2f" % (accuracy_score(y_pred, y_test)))
print("F1 score: %.2f" % (f1_score(y_pred, y_test)))
#Create confusion matrix and normalizes it over predicted (columns)
result = confusion_matrix(y_test, y_pred)
print(result)


919/919 [==============================] - 8s 9ms/step
### Evaluation on test set ###
Accuracy: 1.00
F1 score: 0.99
[[26313    44]
 [   45  2998]]


In [14]:
scores = model.evaluate(X_val, y_val, verbose=0)
y_pred = np.round(model.predict(X_val), 0)

print("### Evaluation on validation set ###")
print("Accuracy: %.2f" % (accuracy_score(y_pred, y_val)))
print("F1 score: %.2f" % (f1_score(y_pred, y_val)))
#Create confusion matrix and normalizes it over predicted (columns)
result = confusion_matrix(y_val, y_pred)
print(result)


644/644 [==============================] - 5s 8ms/step
### Evaluation on validation set ###
Accuracy: 1.00
F1 score: 0.99
[[18206    27]
 [   25  2322]]


## LSTM with Multi Configuration

In [20]:
def get_dataset_split(grid_x, grid_y, idx, test_size = 0.33, window_size = 10, overlap_size = 0):
    def build_df(X_configuration, y_configuration, window_size=window_size, overlap_size=overlap_size, label_treshold = 1):
        stride = window_size - overlap_size
        num_windows = (X_configuration.shape[-1]-window_size)//stride + 1

        windows = np.zeros((X_configuration.shape[0]*(num_windows-1),window_size))
        windows_label = np.zeros((y_configuration.shape[0]*(num_windows-1),window_size), dtype='bool')


        for i in range(X_configuration.shape[0]):
            tmp_windows = np.array([X_configuration[i,j:j+window_size] for j in range(0,stride*num_windows,stride)])
            tmp_windows_labels = np.array([y_configuration[i,j:j+window_size] for j in range(0,stride*num_windows,stride)])
            windows[i*(num_windows-1):(i+1)*(num_windows-1)] = tmp_windows[:-1,:]
            windows_label[i*(num_windows-1):(i+1)*(num_windows-1)] = tmp_windows_labels[1:,:]

        windows_label = np.sum(windows_label, axis=-1)
        windows_label[windows_label<label_treshold] = 0
        windows_label[windows_label>=label_treshold] = 1

        df = pd.DataFrame(windows, columns=[f't_{i}' for i in range(windows.shape[-1])]).sample(frac=1)
        y_df = pd.DataFrame({'future_flare':windows_label})
        df = pd.concat([df, y_df], axis=1)

        return df

    run_test_index = int((1-test_size) * params['run'])

    # build the dataframe
    X_configuration = []
    y_configuration = []
    # for s, t, d, m in tqdm(product(params['sigma'], params['theta'], params['delta'], params['mu'])):
    for s, t, d, m in tqdm(product(params['sigma'], params['theta'], [0.2], params['mu'])):
        ti = params['theta'].index(t)
        mi = params['mu'].index(m)
        si = params['sigma'].index(s)
        di = params['delta'].index(d)
        X_configuration.append(grid_X[:run_test_index, ti, mi, si, di, :])
        y_configuration.append(grid_y[:run_test_index, ti, mi, si, di, :])

    X_configuration = np.hstack(X_configuration)
    y_configuration = np.hstack(y_configuration)
    # df training
    df_train = build_df(X_configuration, y_configuration)

    # build the dataframe
    X_configuration = []
    y_configuration = []
    # for s, t, d, m in tqdm(product(params['sigma'], params['theta'], params['delta'], params['mu'])):
    for s, t, d, m in tqdm(product(params['sigma'], params['theta'], [0.2], params['mu'])):
        ti = params['theta'].index(t)
        mi = params['mu'].index(m)
        si = params['sigma'].index(s)
        di = params['delta'].index(d)
        X_configuration.append(grid_X[run_test_index:, ti, mi, si, di, :])
        y_configuration.append(grid_y[run_test_index:, ti, mi, si, di, :])
    X_configuration = np.hstack(X_configuration)
    y_configuration = np.hstack(y_configuration)
    # df test
    df_test = build_df(X_configuration, y_configuration)

    return df_train, df_test

df_train, df_test = get_dataset_split(grid_X, grid_y, idx, window_size=20, overlap_size=18)

64it [00:00, 100387.23it/s]
64it [00:00, 133483.57it/s]


In [21]:
# number of classes
print('Training set:')
print(df_train['future_flare'].value_counts(), '\n')
print('Test set:')
print(df_test['future_flare'].value_counts(), '\n')

Training set:
0    540001
1     99799
Name: future_flare, dtype: int64 

Test set:
0    270158
1     49742
Name: future_flare, dtype: int64 



In [22]:
# extract X and y from training dataframe
X_train = df_train.drop(['future_flare'], axis=1).to_numpy()
y_train = df_train['future_flare'].to_numpy()

# extract X and y from test dataframe
X_test = df_test.drop(['future_flare'], axis=1).to_numpy()
y_test = df_test['future_flare'].to_numpy()

print('Train shape:',len(X_train), len(y_train))
print('Test shape:  ',len(X_test), '', len(y_test))

Train shape: 639800 639800
Test shape:   319900  319900


In [23]:
print('Train:', X_train.shape, y_train.shape)
print('Test: ', X_test.shape, y_test.shape)

Train: (639800, 20) (639800,)
Test:  (319900, 20) (319900,)


In [24]:
# define model
model = Sequential()
model.add(Bidirectional(LSTM(20, activation='relu'), input_shape=(X_train.shape[1], 1)))
model.add(Dense(30, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1_m, 'accuracy'])

# Calculate the weights for each class so that we can balance the data
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_1 (Bidirectio  (None, 40)               3520      
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, 30)                1230      
                                                                 
 dense_7 (Dense)             (None, 10)                310       
                                                                 
 dense_8 (Dense)             (None, 1)                 11        
                                                                 
Total params: 5,071
Trainable params: 5,071
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
# fit model
model.fit(X_train, y_train, epochs=20, verbose=1)

Epoch 1/20
19994/19994 [==============================] - 453s 22ms/step - loss: 0.1428 - f1_m: 0.7814 - accuracy: 0.9445
Epoch 2/20
19994/19994 [==============================] - 341s 17ms/step - loss: 0.1267 - f1_m: 0.7996 - accuracy: 0.9485
Epoch 3/20
19994/19994 [==============================] - 378s 19ms/step - loss: 0.1220 - f1_m: 0.8041 - accuracy: 0.9501
Epoch 4/20
19994/19994 [==============================] - 405s 20ms/step - loss: 0.1179 - f1_m: 0.8124 - accuracy: 0.9516
Epoch 5/20
19994/19994 [==============================] - 496s 25ms/step - loss: 0.1161 - f1_m: 0.8153 - accuracy: 0.9523
Epoch 6/20
19994/19994 [==============================] - 500s 25ms/step - loss: 0.1147 - f1_m: 0.8178 - accuracy: 0.9530
Epoch 7/20
19994/19994 [==============================] - 454s 23ms/step - loss: 0.1142 - f1_m: 0.8178 - accuracy: 0.9532
Epoch 8/20
19994/19994 [==============================] - 402s 20ms/step - loss: 0.1198 - f1_m: 0.8180 - accuracy: 0.9530
Epoch 9/20
19994/19994 [

In [26]:
scores = model.evaluate(X_test, y_test, verbose=0)
y_pred = np.round(model.predict(X_test), 0)

print("### Evaluation on test set ###")
print("Accuracy: %.2f" % (accuracy_score(y_pred, y_test)))
print("F1 score: %.2f" % (f1_score(y_pred, y_test)))
#Create confusion matrix and normalizes it over predicted (columns)
result = confusion_matrix(y_test, y_pred)
print(result)


9997/9997 [==============================] - 63s 6ms/step
### Evaluation on test set ###
Accuracy: 0.95
F1 score: 0.84
[[266379   3779]
 [ 11261  38481]]
